In [ ]:
import torch

x = torch.tensor([[1,2],[3,4]], dtype=torch.float32)
print("x:" ,x)

import numpy as np
x=torch.tensor(np.array([[1,2],[3,4]]), dtype=torch.float32)
print("x:" ,x)

y = torch.zeros(2,2)
print("y:" ,y)

z = torch.ones(2,2)
print("z:" ,z)

w = torch.rand(2,2)
print("w:" ,w)

In [ ]:
x = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])

print(x.dtype)
print(x.shape)
print(x.device)
print(x.requires_grad)

In [ ]:
print(x)
print(x[1][2])
print(x[0:2,1:3])
print(x[:,2])

print(x.T)

print(x.reshape(1,9))
print(x.reshape(1,-1))

y = x.unsqueeze(0)
print(y)
print(y.shape)

z = x.squeeze()
print(z)
print(z.shape)

In [ ]:
print(x.permute(1,0))#transpose

In [ ]:
tensor_list = [torch.rand(2,3,4) for i in range(10)]
print(torch.cat(tensor_list, dim=1).shape) #按照第一个维度拼接

In [ ]:
tensor_list = [torch.rand(2,3,4) for i in range(10)]
print(torch.stack(tensor_list, dim=0).shape) #增加一个维度拼接

In [ ]:
x = torch.arange(20,dtype=torch.float32).reshape(5,4)
print(x)

print(x.sum())
print(x.sum(axis=0))

In [ ]:
print(x.add(1))
print(x.sub(1))
print(x.pow(2))
x.add_(1)
print(x)

In [ ]:
x = torch.arange(20,dtype=torch.float32).reshape(5,4)
y = x.clone()
x,x+y,x*y

In [ ]:
print(x @ y.T)
print(torch.mm(x,y.T))

In [ ]:
print(x+2)
print(x+torch.tensor([1,2,3,4]))

# torch 的 autograd

In [ ]:
x = torch.tensor([[1.,2.],[3.,4.]],requires_grad=True)
print(x)

y = x*x
print(y)

loss = y.sum()
loss.backward()
print(x.grad) #x.grad = dy/dx = 2*x

In [ ]:
import torch
from matplotlib import pyplot

device = torch.device('cpu')

N,D_in,H,D_out = 64,1000,100,10
x = torch.randn(N,D_in,device=device)
y = torch.randn(N,D_out,device=device)
w1 = torch.randn(D_in,H,device=device,requires_grad=True)
w2 = torch.randn(H,D_out,device=device,requires_grad=True)

learning_rate = 1e-7
losses = []
print("start")
for i in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    loss = (y_pred - y).pow(2).sum()
    losses.append(loss.item())

    if i % 100 == 99:
        print(i,loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

pyplot.plot(losses)

# build a simple neural network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.linear1 = nn.Linear(2,8,bias=False)
        self.linear2 = nn.Linear(8,1)

    def forward(self,x):
        h_relu = F.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred
    
net = SimpleNet()

In [ ]:
print(net)

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(),lr=1e-2)

data = torch.tensor([[1.,2.],[3.,4.]],dtype=torch.float32)
label = torch.tensor([[0.],[1.]],dtype=torch.float32)

for i in range(500):
    outputs = net(data)
    loss = criterion(outputs,label)

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()
   
    if i % 100 == 99:
        print(f"i[{i+1}/{500}], loss:{loss.item():.4f}")